### Shakespearian character-level RNN model

In [4]:
from __future__ import print_function
import sys
import numpy as np
import re
import random
import pickle

from nltk.corpus import gutenberg

from keras.models import Sequential
from keras.layers import Dense, Bidirectional, Dropout
from keras.layers import SimpleRNN, GRU, BatchNormalization

from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [6]:
# first time: download gutenberg with python and nlt.download('gutenberg')
hamlet = gutenberg.words('shakespeare-hamlet.txt')   # Ordered list of words in Hamlet

print(hamlet[:10])

['[', 'The', 'Tragedie', 'of', 'Hamlet', 'by', 'William', 'Shakespeare', '1599', ']']


In [10]:
# lowercase all words

text =''

for word in hamlet:            
    text+=str(word).lower()   
    text+= ' '                 
    
    
print('Corpus length, Hamlet only:', len(text))

Corpus length, Hamlet only: 166765


In [12]:
'''
Break text into :

Features  -    Character-level sequences of fixed length        
Labels    -    The next character in sequence     

'''

training_sequences = []          # Empty list to collect each sequence 
next_chars = []                  # Empty list to collect next character in sequence
seq_len, stride = 35, 1          # the input sequence will be 35 chars long, and the window moves by 1 char each time


for i in range(0, len(text) - seq_len, stride): 
    training_sequences.append(text[i: i + seq_len]) 
    next_chars.append(text[i + seq_len])            

In [13]:
# Print out sequences and labels to verify

print('Number of sequences:', len(training_sequences))
print('First sequences:', training_sequences[:1])
print('Next characters in sequence:', next_chars[:1])
print('Second sequences:', training_sequences[1:2])
print('Next characters in sequence:', next_chars[1:2])

Number of sequences: 166730
First sequences: ['[ the tragedie of hamlet by william']
Next characters in sequence: [' ']
Second sequences: [' the tragedie of hamlet by william ']
Next characters in sequence: ['s']


In [14]:
# Get sorted list of unique characters in hamlet

characters = sorted(list(set(text)))
print('Total characters:', len(characters))

Total characters: 43


In [15]:
# Make lookup dictionaries to map each unique charatcer with an integers
# for training purposes

char_indices = dict((l, i) for i, l in enumerate(characters))
indices_char = dict((i, l) for i, l in enumerate(characters))

print(char_indices)

{' ': 0, '!': 1, '&': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '1': 9, '5': 10, '9': 11, ':': 12, ';': 13, '?': 14, '[': 15, ']': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'k': 27, 'l': 28, 'm': 29, 'n': 30, 'o': 31, 'p': 32, 'q': 33, 'r': 34, 's': 35, 't': 36, 'u': 37, 'v': 38, 'w': 39, 'x': 40, 'y': 41, 'z': 42}


In [16]:
#Create a Matrix of zeros
# With dimensions : (training sequences, length of each sequence, total unique characters)
# 3 dimensions because it's one-hot encoded (double letters do not matter here)
x = np.zeros((len(training_sequences), seq_len, len(characters)), dtype=np.bool)
y = np.zeros((len(training_sequences), len(characters)), dtype=np.bool)


for index, sequence in enumerate(training_sequences):     #Iterate over training sequences
    
    for sub_index, chars in enumerate(sequence):          #Iterate over characters per sequence
        
        x[index, sub_index, char_indices[chars]] = 1      #Update character position in feature matrix to 1
        
    y[index, char_indices[next_chars[index]]] = 1         #Update character position in label matrix to 1

In [17]:
print('Data vectorization completed.')
print('Feature vectors shape', x.shape)
print('Label vectors shape', y.shape)

Data vectorization completed.
Feature vectors shape (166730, 35, 43)
Label vectors shape (166730, 43)


In [18]:
# Simple 1-layered RNN with 128 neurons

def SimpleRNN_model():
    model = Sequential()
    model.add(SimpleRNN(128, input_shape=(seq_len, len(characters))))
    model.add(Dense(len(characters), activation='softmax'))
    return model

In [19]:
# Two stacked RNN layers, both with 128 neurons

def SimpleRNN_stacked_model():
    model = Sequential()
    '''
    return_sequences allows the RNN to output the array of predictions, one per time step,
    instead of only the last prediction. This is useful in stacking RNNs
    return_sequences has to be false on last layer (for classification)
    '''
    model.add(SimpleRNN(128, input_shape=(seq_len, len(characters)), return_sequences=True))
    model.add(SimpleRNN(128))
    model.add(Dense(len(characters), activation='softmax'))
    return model

In [20]:
# Two stacked GRU layers with 128 neurons each

def GRU_stacked_model():
    model = Sequential()
    model.add(GRU(128, input_shape=(seq_len, len(characters)), return_sequences=True))
    model.add(GRU(128))
    model.add(Dense(len(characters), activation='softmax'))
    return model

In [21]:
# Two stacked bi-directional layers with 128 neurons each
# a bidirectinal gru processes the data in both the normal and the reverse sequence
# the output formula is y_t = g(W_y*[a_t, a_rt] + b_y)
# The activation and weight matrices here are simply defined by the model nested within the bi-directional layer.
def Bi_directional_GRU():
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=(seq_len, len(characters))))
    model.add(Bidirectional(GRU(128)))
    model.add(Dense(len(characters), activation='softmax'))
    return model

In [22]:
# Large GRU model with 3 GRU layers and one densely connected hidden layer, using dropout strategy
# the recurrent dropout shuts down the same neurons at each time step, otherwise too much info would be lost

def larger_GRU():
    model = Sequential()
    model.add(GRU(128, input_shape=(seq_len, len(characters)),
                       dropout=0.2,
                       recurrent_dropout=0.2,
                       return_sequences=True))
    
    model.add(GRU(128, dropout=0.2,
                  recurrent_dropout=0.2,
                  return_sequences=True))
    
    model.add(GRU(128, dropout=0.2,
                  recurrent_dropout=0.2))
    
    model.add(Dense(128, activation='relu'))
    
    model.add(Dense(len(characters), activation='softmax'))
    
    return model



In [23]:
# All defined models

all_models = [SimpleRNN_model,
              SimpleRNN_stacked_model,
              GRU_stacked_model,
              Bi_directional_GRU, 
              Bi_directional_GRU,
              larger_GRU]

### Sampling tresholds
By introducing some randomness in the selection of the next character, the model can 'experiment' new routes and come out with new novel sequences, instead of just replicate the ones in the training set.

In [24]:
# Sampling a character index from a probability array
'''
weighted randomness: instead of choosing the char with most probability out
of the softmax, the model could chose even others, with a probability equal to the char 
probability out of the softmax.
The treshold determinates the randomness: the higher, the more random is the output.
'''

    
def sample(softmax_predictions, sample_threshold=1.0):
    
    softmax_preds = np.asarray(softmax_predictions).astype('float64')    # Make array of predictions, convert to float
    
    log_preds = np.log(softmax_preds) / sample_threshold                 # Log normalize and divide by threshold
    
    exp_preds = np.exp(log_preds)                                        # Compute exponents of log normalized terms
     
    norm_preds = exp_preds / np.sum(exp_preds)                           # Normalize predictions
    
    prob = np.random.multinomial(1, norm_preds, 1)                       # Draw sample from multinomial distribution
    
    return np.argmax(prob)                                               #Return max value

In [25]:
# Callback Function executed epoch end, generates Prints 
'''
This function will take a random sequence of characters from the Hamlet text
and then generate 400 characters to follow on, starting from the given input
'''
def on_epoch_end(epoch, _):
    global model, model_name
    
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - seq_len - 1)    # Random index position to start sample input sequence
    end_index = start_index + seq_len                           # End of sequence, corresponding to training sequence length
    
    sampling_range = [0.3, 0.5, 0.7, 1.0, 1.2]                  # Sampling entropy threshold
    
    for threshold in sampling_range:
        print('----- *Sampling Threshold* :', threshold)
        
        generated = ''                                          # Empty string to collect sequence
        
        sentence = text[start_index: end_index]                 # Random input sequence taken from Hamlet
        generated += sentence                                   # Add input sentence to generated
        
        print('Input sequence to generate from : "' + sentence + '"')
        
        sys.stdout.write(generated)                            # Print out buffer instead of waiting till the end
        
        
        for i in range(400):                                   # Generate 400 next characters in the sequence
            
            x_pred = np.zeros((1, seq_len, len(characters)))   # Matrix of zeros for input sentence
            
            for n, char in enumerate(sentence):                # For character in sentence
                
                x_pred[0, n, char_indices[char]] = 1.          # Change index position for character to 1.
                
            preds = model.predict(x_pred, verbose=0)[0]        # Make prediction on input vector
            
            next_index = sample(preds, threshold)              # Get index position of next character using sample function
            
            next_char = indices_char[next_index]               # Get next character using index
            
            generated += next_char                             # Add generated character to sequence (length stays the same)
            sentence = sentence[1:] + next_char
            
            sys.stdout.write(next_char)
            sys.stdout.flush()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [32]:
def test_models(list, epochs=10):
    global model, model_name
    
    for network in list:   
        print('Initiating compilation...')
        
        # Initialize model
        model = network()
        # Get model name
        model_name = re.split(' ', str(network))[1]  
        
        #Filepath to save model with name, epoch and loss 
        filepath = "./models/versions/%s_epoch-{epoch:02d}-loss-{loss:.4f}.h5"%model_name
        
        #Checkpoint callback object 
        checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
        
        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        print('Compiled:', str(model_name))
        
        # Initiate training
        network = model.fit(x, y,
              batch_size=100,
              epochs=epochs,
              callbacks=[print_callback, checkpoint])
        
        # Print model configuration
        model.summary()
           
        #Save model history object for later analysis
        with open('./models/history/%s.pkl'%model_name, 'wb') as file_pi:
            pickle.dump(network.history, file_pi)
            

In [33]:
test_models(all_models, epochs=5)

Initiating compilation...
Compiled: SimpleRNN_model
Epoch 1/5
166730/166730 [==============================] - 15s 91us/step - loss: 2.2316
----- Generating text after Epoch: 0
----- *Sampling Threshold* : 0.3
Input sequence to generate from : "clown . giue me leaue ; heere lies "
clown . giue me leaue ; heere lies , and the mand , the sould the sing , in this loue tha ke the pare , the sithe se the the king , the her . in the wath the south you mast ou he wert , and will my lord , the tha la de me the kenge mad the se the lathe se the me me the me loue the se be the singe , and the se the the me the lathes make the the haul the seall our . what the this se for in the me hea . so the healle , whe s aple , th----- *Sampling Threshold* : 0.5
Input sequence to generate from : "clown . giue me leaue ; heere lies "
clown . giue me leaue ; heere lies , ind dor . ald don . in an the par ham . the hail bot vy lo m it the sing , my lord , mo vs are the me , ard and the llours ond his the mant ,

 the poor be flatter ' d ? no , let endry faidint acr bed , es murtiue nor slather sayees do natel s ay my sagl wit vply mosh diy mald east ( pely a ' or ' s heay : vsto king . qu . ace til in tiot vbteunitw pow , sfeacowas whithtinoughne : as now : teee spyel ' r brawitkence oth onines thitkry watt bed me -eles detitell abr kants you  ofr it letewes & sint bebdull to : irw hrselild sep hes fay you hear gathard , and siscestll cetrEpoch 4/5
166730/166730 [==============================] - 15s 89us/step - loss: 1.7819
----- Generating text after Epoch: 3
----- *Sampling Threshold* : 0.3
Input sequence to generate from : "herod ' s herod . pray you auoid it"
herod ' s herod . pray you auoid it is his fore , and my lord , and it he well is not be well . the hambet the the hame the well my lord , with the winde , and gite the hamlet ham . i haue the winds , and he heare , and the pare , and in the hame the peather his shall the beane , the weare , and ham . the the hame the hame the hame t

 higher rate , then a command to parte the for . wher as ing fere , sthe poll , him . i wile , and me a houre on make ; a the hamtese . now that heruense ; you selor what : it thice io i the make of the fente , be lis mill the . win the noppllay hersing winkengs of ham . i thas that to me siater , and you doous thy sorond he be fiols : on ' thed this thy feare , the make in the parse seluer i whoue sham . that , sham . why thau ; i----- *Sampling Threshold* : 1.0
Input sequence to generate from : " higher rate , then a command to pa"
 higher rate , then a command to pal gome the forre . lee : his of tike mowhuli ster whawhttente an , not tithing anding , hamyeu ' d shy shauen the weimer se vikim . thes are . be pe bthe , at hor our leing goobd ; o thom thebs arm to - pukmce to thre makte fan him s awle ; sheadest , hamlea troust , mrigkengos lor ) fake ; anz theue hherme tit ham . pke pilas you siplstime plr scisens , if nelle daud , to thor wonthelle ie hornes----- *Sampling Threshold

s ; if this be otherwise , if circuing , and more not the part , and let me to sist not in the dome , and loue for owne a like the night , the bland the downe , for he this was my lord , and gard , the priches he man is this part , that ' s to the will the wance , and the pold polition , the polse in the propers , and not this in the word ham . this the manter , the with a crow , so make this more , what mar . the ranger ' d , i wi----- *Sampling Threshold* : 0.7
Input sequence to generate from : "s ; if this be otherwise , if circu"
s ; if this be otherwise , if circunt in beare the tirlon , will is be , and to this hatthith , i speene , one the gant you this parching : but king , the parting do say cannet tay isw sing indeeme of deere of this natthis ophe . so make , would this nempins , that day to the in some so shall so me ? it your fat , with fare , exe will is no word , the crownous of nermes , ald say me sir . and fanderall forme : for be vne the thinke----- *Sampling Threshold

e in many places , giues me superfle ' t apclits gauge : a farreroun of ' seame my not ? hom hincar . it neale . lost : go . let heakes , hat not hay doxatis oud mast aloods tol . no sween as lnowsteflend hownes , i ray gat , wheliost to him ; enfor my dorchaury not mue thiise dimcongruch ; that spill vgall this mintrescson . ercace : eaurae hag saue ; our mor. laudt of heaueties and fellinitous pull ; and thipke intiofach onterperEpoch 2/5
166730/166730 [==============================] - 65s 391us/step - loss: 1.6881
----- Generating text after Epoch: 1
----- *Sampling Threshold* : 0.3
Input sequence to generate from : "he trumpets speake , the trumpet to"
he trumpets speake , the trumpet to this from , that mar . i haue a sinde , what i haue this a mad , that i haue a same , and the dist , the mad so , and the spolles of ham . the mad . i haue with , the chart , and martious , the a seare : saue i will the king . i am i the seate , who shall to the prosing . i will me , and i haue a 

...

at the heeles of this mothers admiration the preceaue , the sent out of the procentare the heauen , and the sent me that i haue not the weach , what shall the thinke the polenter . oh the players , why the prepently of the preceare the polonius . made the selfe : oh that i do not the courtiue , the poyson , that i haue seene in the sent a past : and the preceaue the watch of the true . he was a strange the carriages , and the prece----- *Sampling Threshold* : 0.5
Input sequence to generate from : "at the heeles of this mothers admir"
at the heeles of this mothers admiration : now , and be the please me the hath string that i pray you see : the poysonesse with for you me : why see the weach , and he beare what we doe the saies , on the precounter , for the polon . that i cannot shall be shall not the procert : heauen , as the prepently the true and made , that it be the morne , if the points of the body . what , what is the seate ? ile prosent the course the sel----- *Sampling Threshold

gnorance . go too , ile no more on the , not , ioys couson , forlowes be sposte at tak ' d now sinde liung to the saue i know vncome ; for whose way ' dard : like it in his say ) alo a polower , and thy phapt prroent saite , heere now the kint . good off to the necell kide the frightrue vs ad heere tome thinres , how ? how she can the sanfusion , os , as the our haue a what of like our boted his ratten ) rescall , home masoure moth----- *Sampling Threshold* : 1.2
Input sequence to generate from : "gnorance . go too , ile no more on "
gnorance . go too , ile no more on father heere to handerserh , inhinke daiesblaces remaine of your calnes . e anwast asperace , i ' th ' be souds and this wich sot : tiue , and shrae hou him s?re mumthexridleftued now beritins tia , as the aul ' all : posth faithing outnouen high to horatiuellposse like in soou to shilt. sound once of if pit ont ( i kh ile fit you opes vollasity ham . the corwimall ? ? is diueltile , nor potkilanbEpoch 3/5
166730/166730 [

r that which thou hast done , must friends of the passion in his our more and be ; hor . ham . ought the head ? hor . i haue euen him hamlet . her both : good my lord , i marcellus . vertue fooll , and againe not that vpon him , mar . marke you thinke of our then king ? question , for there , the good out are friend ham . my lord ham . a pilefters ) like you a montant : he from the king ? the king hollow thou ham . at after barnd t----- *Sampling Threshold* : 1.0
Input sequence to generate from : "r that which thou hast done , must "
r that which thou hast done , must first by triuill in they are finde ! hor . ylaches my decillame , and nothing leaue violet . for the hath thou such to your england amborty bore ham . i would heere him , dine heere , how on a poune ham . goe in ham . what i do my peare ? ham . from his mine a poor? ham . then footedy my demin . o ' re , my clay there of all for my top . let time ? her i ? what haue mercily , how ' d ! we haue it ----- *Sampling Threshold

that thou so many princes , at a shall make the mand the polon . and the can the prasing , and the greath , and the could , and the grasion , and which , and will prosing , what will shall what haue the polon . what i haue see , and what i haue be in his and in the come and the poron . i haue speech , what his lord . what i passion all be ham . what shall , the this sould , and the grie , and the king , and what will more the come ----- *Sampling Threshold* : 0.5
Input sequence to generate from : "that thou so many princes , at a sh"
that thou so many princes , at a shall the lord good my shall , and the what i haue him . what is the mad a pace , and shall to the the king . what is make make come , and the not from light , the from , and the my lord , and i haue my lord , and the but dis in all whild the fare to speepe , or the not and all , the will not and ham . oh the read shall the come , and which the king . marring , or she there the tame and dase , and a----- *Sampling Threshold

aue , who was in life , a foolish poile branders ham . lord , i cun no wan so your ruck ' qu . fand hyeromlets thing doop - blocke here connt to vison ' d ? polonot , but vnt to i by told . the cqu . thou duill brfainass veriges ham . i sell merter the driquand , whift giue hore ( and w'lke crrath with ,ofer my handd gen whose bost or shoulde ; not will saue , and in fardent ? anow it you wild tiue and passreamt gentlafy : we spont_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 35, 128)           66048     
_________________________________________________________________
gru_8 (GRU)                  (None, 35, 128)           98688     
_________________________________________________________________
gru_9 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_9 (Dense)              (None, 1